In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision import datasets
from torch.utils.data import DataLoader


In [3]:
data_path = './data/'
save_path = './parameter/'
epochs = 30
batch_size = 64
learning_rate = 0.001

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
train_transform = transforms.Compose([
    transforms.Resize(32),
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(30),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))]
)

test_transform = transforms.Compose([
    transforms.Resize(32),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))]
)

In [7]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

train_data = datasets.CIFAR10(
    root=data_path,
    train=True,
    download=True,
    transform=train_transform
)
test_data = datasets.CIFAR10(
    root=data_path,
    train=False,
    download=True,
    transform=test_transform
)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data/
Files already downloaded and verified


In [8]:
trainloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
testloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [9]:
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.fc = nn.Linear(8*8*32, num_classes)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        flatten = out.view(out.size(0), -1)
        score = self.fc(flatten)
        return score

In [10]:
model = ConvNet()
model = models.resnet18(weights='DEFAULT')
num_classes = 10
model.fc = nn.Linear(512, num_classes)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [11]:
def train(epoch):
    print('\nEpoch: %d'%epoch)
    # model train mode로 전환
    model.train()
    running_loss = 0.0
    running_acc = 0.0
    total = 0
    for (inputs, labels) in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        _, pred = torch.max(outputs, 1)
        total += outputs.size(0)
        running_acc += (pred == labels).sum().item()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    total_loss = running_loss / len(trainloader)
    total_acc = 100 * running_acc / total
    print(f'Train epoch : {epoch} loss : {total_loss} Acc : {total_acc}%')

In [12]:
def test(epoch):
    print('\nEpoch: %d'%epoch)
    # model eval mode로 전환
    model.eval()
    running_loss = 0.0
    running_acc = 0.0
    total = 0
    with torch.no_grad():
        for (inputs, labels) in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, pred = torch.max(outputs, 1)
            total += outputs.size(0)
            running_acc += (pred == labels).sum().item()
            loss = criterion(outputs, labels)
            running_loss += loss.item()
        total_loss = running_loss / len(testloader)
        total_acc = 100 * running_acc / total
        print(f'Test epoch : {epoch} loss : {total_loss} Acc : {total_acc}%')

In [13]:
for epoch in range(epochs):
    train(epoch)
    test(epoch)
    path = os.path.join(save_path, f'epoch_{epoch}.pth')
    torch.save(model.state_dict(), path)


Epoch: 0
Train epoch : 0 loss : 2.2775159903499476 Acc : 15.33%

Epoch: 0
Test epoch : 0 loss : 2.235907741412995 Acc : 20.44%

Epoch: 1
Train epoch : 1 loss : 2.175062928358307 Acc : 23.716%

Epoch: 1
Test epoch : 1 loss : 2.1026173017586873 Acc : 26.2%

Epoch: 2
Train epoch : 2 loss : 2.0678059127934447 Acc : 27.116%

Epoch: 2
Test epoch : 2 loss : 2.0184981094044483 Acc : 28.85%

Epoch: 3
Train epoch : 3 loss : 1.9906947208792352 Acc : 30.042%

Epoch: 3
Test epoch : 3 loss : 1.9476255459390628 Acc : 30.93%

Epoch: 4
Train epoch : 4 loss : 1.921479790259505 Acc : 32.532%

Epoch: 4
Test epoch : 4 loss : 1.892221020285491 Acc : 32.83%

Epoch: 5
Train epoch : 5 loss : 1.8705830717330698 Acc : 34.022%

Epoch: 5
Test epoch : 5 loss : 1.8482512197676737 Acc : 34.62%

Epoch: 6
Train epoch : 6 loss : 1.8316389714055659 Acc : 35.568%

Epoch: 6
Test epoch : 6 loss : 1.8028171943251494 Acc : 36.27%

Epoch: 7
Train epoch : 7 loss : 1.7895599862803584 Acc : 37.078%

Epoch: 7
Test epoch : 7 loss 